In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import struct

In [2]:
# generate a,b
def generate_vectors():
    a_uniform_distribution = np.random.uniform(low=-10_0000, high=10_0000, size=(1, 768)) #9.2E-41~3.38E38
    b_uniform_distribution = np.random.choice([0, 1], size=(768, 1))

    return a_uniform_distribution, b_uniform_distribution

In [3]:
#base
def multiply_vectors(a,b):
    a_bfloat16 = a.astype(np.float16)
    result = np.dot(a_bfloat16, b)
    return result

In [4]:
#向量中元素转化为二进制操作
def Modify_tensor(tensor):
    bf_list = tensor.detach().numpy().tolist() # 转成numpy变量再转换成list    
    binary_array = []

    for bf_nums in bf_list:
        for bf_num in bf_nums:
            float_bytes = struct.pack('>f', bf_num)
            byte_array = ''.join(f'{byte:08b}' for byte in float_bytes)
            binary_array.append(byte_array)
    return binary_array

In [11]:
#ReDCIM
def ReDCIM_multiply(a,b):
    a_tensor = torch.from_numpy(a)
    b_tensor = torch.from_numpy(b)
    a_fp32 = a_tensor.float()
    a_modified = Modify_tensor(a_fp32)

    ##预对齐
    a_exp = []
    for number in a_modified:
        extracted_part = number[1:9]
        a_exp.append(extracted_part)
    a_exp_int = [int(binary, 2) for binary in a_exp]
    a_exp_max = max(a_exp_int)
    a_difference = [a_exp_max - value for value in a_exp_int]

    a_mantissa = []
    for number in a_modified:
        if number[0] == '1':
            inverted_part ='0' + ''.join('1' if bit == '0' else '0' for bit in number[9:15]) #取反码
            complement = format(int(inverted_part, 2) + 1, '07b') #取补码
            extracted_part = number[0] + complement #加符号位
        else:
            extracted_part = number[0] + '1' + number[9:14]
        a_mantissa.append(extracted_part)

    a_shifted_mantissa_values = []
    for mant, diff in zip(a_mantissa, a_difference):
        sign = int(mant[0], 2) #将mantissa 的首位取为符号位，并转化为int类型
        if diff > 0:
            shifted_value_binary = (sign * '1' if sign == 1 else '0') * diff + mant[:-diff] #str类型拼接
        else:
            shifted_value_binary = mant
        a_shifted_mantissa_values.append(shifted_value_binary)

    #尾数和weight 相乘
    product_mantissa = [int(a, 2)  * b for a, b in zip(a_shifted_mantissa_values, b)]
    sum_product_mantissa = sum(product_mantissa)
    sum_product_mantissa_binary = format(sum_product_mantissa,'017b') #768个8bit数相加，最大17位
    sign = 1 if sum_product_mantissa < 0 else 0

    if sign == 1:
        mantissa_all = ~int(sum_product_mantissa, 2) + 1
    else:
        mantissa_all = sum_product_mantissa_binary[1:]

    combined_result = ((-1) ** int(sign)) * (2 ** (a_exp_max - 127)) * mantissa_all
    return combined_result

In [12]:
#compare
def compare_and_calculate_y(results_a, results_b):
    differences = []
    for result_standard, ReDCIM in zip(results_a, results_b):
        difference = abs(torch.abs(result_standard - ReDCIM)/result_standard)
        differences.append(difference.item())

    mean_difference = np.mean(differences)
    return mean_difference

def run_calculations_with_actual_implementation(n):
    results_standard = []
    results_ReDCIM = []
    
    for _ in range(500):
        a, b = generate_vectors()
        result_standard = multiply_vectors(a, b)
        result_ReDCIM = ReDCIM_multiply(a, b)

        results_standard.append(result_standard)
        results_ReDCIM.append(result_ReDCIM)

    ReDCIM_main_difference = compare_and_calculate_y(results_standard, results_ReDCIM)
    return ReDCIM_main_difference

ReDCIM_main_difference = run_calculations_with_actual_implementation(5000)[0]

print("ReDCIM 平均误差 (normalized) " + str(ReDCIM_main_difference))


C:\Users\YJQ\AppData\Local\Temp\ipykernel_7716\3632802187.py:3: RuntimeWarning: overflow encountered in cast
  a_bfloat16 = a.astype(np.float16)


TypeError: unsupported format string passed to numpy.ndarray.__format__